<a href="https://colab.research.google.com/github/Fashad-Ahmed/The-Speako/blob/trancription-branch-tash-v1/zenML_Pipeline_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install zenML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 795 kB 5.4 MB/s 
     |████████████████████████████████| 181 kB 42.6 MB/s 
     |████████████████████████████████| 67 kB 3.6 MB/s 
     |████████████████████████████████| 1.3 MB 51.3 MB/s 
     |████████████████████████████████| 95 kB 1.9 MB/s 
     |████████████████████████████████| 96 kB 5.2 MB/s 
     |████████████████████████████████| 10.9 MB 40.1 MB/s 
     |████████████████████████████████| 636 kB 43.8 MB/s 
     |████████████████████████████████| 235 kB 38.1 MB/s 
     |████████████████████████████████| 11.1 MB 39.6 MB/s 
     |████████████████████████████████| 561 kB 56.4 MB/s 
     |████████████████████████████████| 40 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 
     |████████████████████████████████| 147 kB 52.9 MB/s 
     |████████████████████████████████| 135 kB 60.7 MB/s 
     |████████████████████████████████

In [1]:
!zenml init

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.11.0
Creating default profile...
Initializing profile default...
Registering default stack...
⠋ Initializing ZenML repository at /content.
Registered stack component with type 'orchestrator' and name 'default'.
Registered stack component with type 'metadata_store' and name 'default'.
Registered stack component with type 'artifact_store' and name 'default'.
Registered stack with name 'default'.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
Created and activated default profile.
ZenML repository initialized at /content.
⠧ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.

The loc

In [4]:
!zenml integration install sklearn -y # code is not workig with out integrating with some libraires|

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠏ Installing integrations...


In [5]:
from zenml.pipelines import pipeline
from zenml.steps import step, Output


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [6]:
from zenml.materializers.base_materializer import BaseMaterializer
from zenml.artifacts import DataArtifact

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [9]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile  

     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 153 kB 60.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [10]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config
from wav2vec2 import CTCLoss
import soundfile as sf

In [11]:
@step
def upload_audio_file() -> Output(path= str):
  path="/content/amanda_gorman.flac"
  return path


In [16]:
# def asr_transcript(audio_file: str,
#                    model_transcribe:Wav2Vec2ForCTC,
#                    tokenizer_transcribe:Wav2Vec2Tokenizer,
#                    ) -> Output(transcript= str):

@step
def asr_transcript(audio_file: str) -> Output(transcript= str):

      tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
      model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
      transcript = ""

      # Stream over 20 seconds chunks
      stream = librosa.stream(
          audio_file, block_length=20, frame_length=16000, hop_length=16000
      )

      for speech in stream:
          if len(speech.shape) > 1:
              speech = speech[:, 0] + speech[:, 1]

          input_values = tokenizer(speech, return_tensors="pt").input_values
          logits = model(input_values).logits

          predicted_ids = torch.argmax(logits, dim=-1)
          transcription = tokenizer.decode(predicted_ids[0])
          transcript += transcription.lower() + " "
      print(transcript)     
          
      return transcript

In [ ]:
# asr_transcript.entrypoint(audio_file="/content/Wave_files_demos_Welcome (5).wav")

In [17]:
@pipeline
def model_1(step_1,step_2):
    """Links all the steps together in a pipeline"""
    path=step_1()
    transcription=step_2(path)

In [19]:
model_1(
    step_1=upload_audio_file(),
    step_2=asr_transcript(),
).run(run_name="run_1")

mister president doctor byden madam vice president mister mhof americans and the world when day comes we ask ourselves where can we find light in this never ending shade the loss we carry a sea we must wade we've braved the belly of the beast we've learned that quiet isn't always peace in the norms and notions of what just is isn't always just is and yet the on as ours before we knew it somehow we do it somehow we've weathered and witnessed a nation that isn't broken but simply unfinished we the successors of a country and a time were a skinny black girl descended from slaves and raised by a single mother can dream of becoming president only to find herself reciting for one and yes we are far from polished far from pestine but that doesn't mean we are striving to form a union that is perfect we are striving to forge our union with purpose to compose a country committed to all cultures colors characters and conditions of man and so we lift our gaze not to what stands between us but what

In [ ]:
# "@pipeline"
# def model_2(step_1,step_2,step_3,step_4):
#     """Links all the steps together in a pipeline"""
#     path=step_1()
#     model=step_2().with_return_materializers(MyMaterializer_model)
#     token=step_3().with_return_materializers(MyMaterializer_token)
#     transcription=step_4(path,model,token)
    


In [ ]:
# class MyMaterializer_token(BaseMaterializer):
#     ASSOCIATED_TYPES = (Wav2Vec2Tokenizer,)
#     ASSOCIATED_ARTIFACT_TYPES = (DataArtifact,)
#     def handle_input() -> Wav2Vec2ForCTC:
#         """Read from artifact store"""
#         tokenizer_transcribe = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
#         return tokenizer_transcribe

In [ ]:
# class MyMaterializer_model(BaseMaterializer):
#     ASSOCIATED_TYPES = (Wav2Vec2ForCTC,)
#     ASSOCIATED_ARTIFACT_TYPES = (DataArtifact,)
#     def handle_input() -> Wav2Vec2ForCTC:
#         """Read from artifact store"""
#         model_transcribe = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
#         return model_transcribe


In [56]:
# model_2(
#     step_1=upload_audio_file(),
#     step_2=model_load().with_return_materializers(MyMaterializer_model),
#     step_3=token_load().with_return_materializers(MyMaterializer_token),
#     step_4=asr_transcript(),
# ).run(run_name="run_3")